In [3]:
!pip install sounddevice wavio


   ---------------------------------------- 2/2 [sounddevice]



In [11]:
!pip install resampy

  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.1 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.1 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.1 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.1 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.1 MB 305.2 kB/s eta 0:00:09
   ------ --------------------------------- 0.5/3.1 MB 305.2 kB/s eta 0:00:09
   ------ --------------------------------- 0.5/3.1 MB 305.2 kB/s eta 0:00:09
   ------ --------------------------------- 0.5/3.1 MB 305.2 kB/s eta 0:00:09
   

In [1]:
import resampy
print("resampy installed correctly")

resampy installed correctly


In [15]:
import tkinter as tk
from tkinter import filedialog, messagebox
import librosa
import numpy as np
import sounddevice as sd
import scipy.io.wavfile as wav
from keras.models import load_model

# Load the trained CNN model
model = load_model("voice_emotion_model.keras")

# Correct order of emotion classes (from training)
class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']

# Extract features function (for 40x200 shape)
def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=22050)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        if mfccs.shape[1] < 200:
            pad_width = 200 - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :200]
        mfccs = mfccs.reshape(1, 40, 200, 1)
        return mfccs
    except Exception as e:
        print("Error extracting features:", e)
        return None

# Predict emotion from audio file
def predict_emotion(file_path):
    features = extract_features(file_path)
    if features is None:
        messagebox.showerror("Error", "Could not process the audio file.")
        return
    prediction = model.predict(features)
    predicted_index = np.argmax(prediction)
    emotion = class_labels[predicted_index]
    messagebox.showinfo("Prediction", f"Predicted Emotion: {emotion}")

# Browse file handler
def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("WAV Files", "*.wav")])
    if file_path:
        predict_emotion(file_path)

# Voice recording and prediction
def record_voice():
    try:
        duration = 3  # seconds
        fs = 22050
        messagebox.showinfo("Recording", "Recording for 3 seconds...")
        recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
        sd.wait()
        wav.write("recorded_audio.wav", fs, recording)
        predict_emotion("recorded_audio.wav")
    except Exception as e:
        messagebox.showerror("Error", f"Recording failed: {e}")

# GUI Setup
root = tk.Tk()
root.title("🎧 Voice Emotion Detection")
root.geometry("400x300")

tk.Label(root, text="Voice Emotion Detection", font=("Helvetica", 16)).pack(pady=20)
tk.Button(root, text="Upload Audio File", command=browse_file, width=25, bg="lightblue").pack(pady=10)
tk.Button(root, text="🎙 Record Your Voice", command=record_voice, width=25, bg="lightgreen").pack(pady=10)
tk.Button(root, text="Exit", command=root.destroy, width=25, bg="salmon").pack(pady=10)

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
